# Little experiment in sentiment classification

* Pick a large number of sentences that end with `:)` or `:(`
* Train a classifier to try to distinguish among them
* Pick the classifier's brain to see which are the distinguishing features

# Step 1: Data preparation

* Let us query for `:)` and `:(` and grab some data into files
* We must make sure to remove the emoticons themselves from the documents
    * **Why?**
    

In [63]:
%%bash

python api_query.py -d 'Suomi24' --retmax 10000 '":)"' > happy_s24.conllu
python api_query.py -d 'Suomi24' --retmax 10000 '":("' > unhappy_s24.conllu


# Step 2: Preprocessing

* Get rid of the emoticons
* Get rid of empty lines
* Turn everything into lines of text, one per sentence

In [64]:
%%bash

# grep -v removes matching rows (opposite to bare grep)
# conllu2text makes one sentence per row
# '^$' is a pattern for empty row, grep -v removes it
# head prints first 10 lines of whatever you give it
# echo prints a message
cat happy_s24.conllu | grep -v ':)' | python conllu2text.py -f LEMMA | grep -v '^$'  > happy_s24.txt
cat unhappy_s24.conllu | grep -v ':(' | python conllu2text.py -f LEMMA | grep -v '^$'  > unhappy_s24.txt
echo "   ****** Happy   *******"
head happy_s24.txt
echo "   ****** Uhanppy *******"
head unhappy_s24.txt


   ****** Happy   *******
moni ihminen kaipailla tänään , niin minä kaivata rakkaus elämä
mä kyllä Deturin entinen asiakas naurattaa
huomata muuten , että mä FB : n kansi#kuva olla jokin sama , kun se paikka , joka nähdä
olla se sellanen aurinkoinen päivitys
niinpä ..
Ok .
Kittielle kiitos kiitos
ja semmonen vielä että se lehdeton olla marsu herkku
näköjään suhde#epäily nostaa taas pää
mikään ei kuiteskaan olla täällä päässä kulahtaa ei olla myöskään elähtäny
   ****** Uhanppy *******
Kumpa pikkuinen alkaa taas vähän rauhoittua
ihme nakutus kuulua nyt , jokin peli tai kone , joka yö ainakin kolme asti
olla itse löysä ., mies ei tykätä
nainen mutta nyt hän ei enää olla työ kun eräs henkilöstö vuokraus firma vähentää väki
palsta ei enää täyttää tarkoitus
ei olla kukaan ja ei olla kikkiä
ei olla
no joo olla se väärin mut mikä se mahtaa
tuo hapatoiva sepustus antaa vain hyvä kuva mikä mummu korva välissä olla ; viha uskova ja se kohtaan joka kannattaa kristillisä perus#arvo
vai , että sama

# Step 3 - train the classifier

In [65]:
import codecs
# Read the two files in Python
happy=map(unicode.strip, codecs.open("happy_s24.txt","r","utf8").readlines())
unhappy=map(unicode.strip, codecs.open("unhappy_s24.txt","r","utf8").readlines())

#How many did we get?
print len(happy), "happy examples"
print len(unhappy), "unhappy examples"

10660 happy examples
8159 unhappy examples


In [58]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.cross_validation import train_test_split

# Turn the text into vectors that can be handled by the classifier
vectorizer=TfidfVectorizer()
data_matrix=vectorizer.fit_transform(happy+unhappy)
# Give each document the target to be predicted
labels=[":)"]*len(happy)+[":("]*len(unhappy)

# Keep 20% of the data aside for testing
data_train,data_test,labels_train,labels_test=train_test_split(data_matrix, labels, test_size=0.2, random_state=0)

# Train the classifier on our data
# C is an important parameter: the smaller, the fewer features will be used
classifier=LinearSVC(C=0.05,penalty="L1",dual=False)
classifier.fit(data_train,labels_train)

# And test its accuracy
print "Accuracy=%.2f%%"%(classifier.score(X_test,Y_test)*100.0)


Accuracy=72.05%


# Step 4: Pick the classifier's brain

* List features with extremely high (associated with positive labels) or extremely low (associated with negative labels) weights

In [67]:
# Print sorted by weight
f_names=vectorizer.get_feature_names()
sorted_by_weight=sorted(zip(classifier.coef_[0],f_names))
# Thirty lowest
for f_weight,f_name in sorted_by_weight[:30]:
    print f_name, f_weight
print "------------------------"
# Thirty highest
for f_weight,f_name in sorted_by_weight[-30:]:
    print f_name, f_weight

ikävä -1.65595850566
valitettavasti -1.6543497579
ei -1.60435502889
surullinen -1.57535242683
harmi -1.5440953811
enää -1.52270343786
linkittää -1.40424162675
ku -1.38572544054
joulu -1.18803347064
gt -1.17967630573
pelottaa -1.11154470212
mut -1.0870789438
peljätä -1.05563467731
tuntua -1.03099360532
joutua -1.02617813819
voi -1.02112936393
huono -1.0137118077
harmittaa -0.970605219623
mutta -0.966156193311
kuolla -0.963548248492
mä -0.928661033032
auttaa -0.828028862144
jäädä -0.805290693775
paha -0.787616167436
kun -0.724629516498
kamala -0.712055124427
sairas -0.693497895989
että -0.681729893822
mennä -0.626297958452
sen -0.60999723854
------------------------
yö 0.151515018012
tykätä 0.160020334218
ryssä 0.181119967931
muu 0.215927541899
kiittää 0.219757629872
tarkoittaa 0.240287228188
vastaus 0.244068271064
hauska 0.254159296564
sinä 0.266858513774
tarvita 0.267891240611
maa 0.279182715177
kiva 0.282005040801
tai 0.283502397026
sielu 0.390673281397
onneksi 0.404332333042
hyvin 0.

# Step 5: what have we got?

* You tell me. :)

# Now you try for yourself

* I packed all the Python parts into the program `inf_features.py` which takes two files, one with positive examples, and one with negative examples, and lists for you the informative features. The examples are one per line, sentences.

In [69]:
%%bash

python inf_features.py -n 5 -c 0.05 happy_s24.txt unhappy_s24.txt

10660 examples of class 1
8159 examples of class 2
Accuracy=67.61%
hyvä -1.34730566106
kiitos -1.12922710688
mukava -0.857320778375
onni -0.776294498314
sä -0.731208835058
------------------------
harmi 1.2812311198
ku 1.3074499882
ikävä 1.45702490104
gt 1.53586089618
ei 1.81224650642


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


 We have other Finnish corpora available as well
    - `ff` 1 million sentences from Futis Forum messages
    - `Fi-Parsebank-50M` 50 million sentences from the Internet
    - `Suomi24` 10 million sentences from Suomi24
* Can you see any difference between these for the same task?
* Can you think of other document classes which you could try?
